## データのスクレイピング

In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re
from typing import List, Dict, Any

In [ ]:
# ----------------------------------------------------------------------
# 1. 定数とデータベース設定/初期化
# ----------------------------------------------------------------------

# データベースのファイル名
DATABASE_NAME = 'github_google_repos.db'
# サーバー負荷軽減のための待機時間（秒）。GitHubへの連続アクセスを防ぐための必須要件。
SLEEP_TIME = 1

def setup_database():
    """SQLiteデータベースをセットアップし、テーブルを作成する"""
    # データベースに接続（ファイルが存在しない場合は新規作成される）
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    
    # 既存のテーブルを削除し、新しく作成する (実行ごとにDBを初期化)
    cursor.execute("DROP TABLE IF EXISTS repositories")
    cursor.execute("""
        CREATE TABLE repositories (
            id INTEGER PRIMARY KEY AUTOINCREMENT, -- 主キー (自動連番) 
            name TEXT NOT NULL,                  -- リポジトリ名 (必須) 
            language TEXT,                       -- 主要な言語 
            stars INTEGER                        -- スターの数 (整数型で保存) 
        )
    """)
    # 変更を確定（コミット）
    conn.commit()
    conn.close()
    print(f"✅ データベース '{DATABASE_NAME}' を初期化しました。")

# ----------------------------------------------------------------------
# 2. スクレイピング関数
# ----------------------------------------------------------------------

def scrape_google_repos() -> List[Dict[str, Any]]:
    """
    リポジトリが見つからなくなるか、リダイレクトが発生するまで、GitHubのGoogleリポジトリを全てスクレイピングする。
    """
    base_url = "https://github.com/google?page="
    all_repos_data = []
    page = 1 # 最初のページから開始
    # 最後にデータが取得された（またはリダイレクトがなかった）ページの番号を保持する (無限ループ対策)
    last_successful_page = 0 

    print(f"▶️ GitHub (Googleアカウント) から全リポジトリ情報をスクレイピング中...")

    # リポジトリがなくなるまで無限ループでページネーションを続ける
    while True:
        url = f"{base_url}{page}"
        print(f"    - ページ {page} ({url}) にアクセス...")
        
        try:
            # HTTPリクエストを送信
            response = requests.get(url)
            # 4xx, 5xxエラーをチェック
            response.raise_for_status() 
            
            # 【⭐ ページネーション終了チェックの再強化：リダイレクト後のページ番号固定チェック ⭐】
            
            # response.url から page=〇〇 のページ番号を正規表現で抽出
            final_page_match = re.search(r'page=(\d+)', response.url)
            # パラメータが見つからない場合（1ページ目など）は1ページ目と見なす
            final_page = int(final_page_match.group(1)) if final_page_match else 1
            
            # リダイレクトチェックの強化: 最終ページが前回成功したページと同じ場合
            # 同じページを繰り返し読み込んでいると判断し、無限ループを防止する。
            if final_page == last_successful_page and last_successful_page != 0:
                print(f"    ⚠️ リダイレクトによる繰り返しを検知しました。最終ページ {final_page} で停止します。")
                print(f"    ✅ 全ページ完了です。")
                break

            # BeautifulSoupでHTMLを解析
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # リポジトリ一覧の要素を全て取得 ('div.source' は個々のリポジトリカードのセレクタ)
            repo_items = soup.select('div.source')

            if not repo_items:
                # リポジトリの要素が一つも見つからなかった場合も、最終ページ到達と判断
                print(f"    ✅ ページ {page} でリポジトリが見つかりませんでした。全ページ完了です。")
                break
                
            # 成功した場合、最終的なページ番号を記録
            last_successful_page = final_page

            # 各リポジトリから必要な情報を抽出
            for item in repo_items:
                # 1. リポジトリ名: <h3>内の<a>タグのhref属性から末尾の要素を取得
                name_tag = item.select_one('h3 a')
                name = name_tag['href'].split('/')[-1] if name_tag and name_tag.has_attr('href') else "N/A"
                
                # 2. 主要な言語: itemprop="programmingLanguage" の<span>タグのテキストを取得
                language_tag = item.select_one('span[itemprop="programmingLanguage"]')
                language = language_tag.text.strip() if language_tag else "Other"
                
                # 3. スターの数: "/stargazers" にリンクしている<a>タグのテキストを取得
                star_tag = item.select_one('a[href$="/stargazers"]')
                stars_text = star_tag.text.strip() if star_tag else "0"

                # 【スター数の変換ロジック】: '52.6k' や '52,600' を整数に変換
                stars_text = stars_text.replace(',', '').lower()
                stars = 0
                if 'k' in stars_text:
                    try:
                        # 'k' を削除し、floatに変換後、1000倍して整数にする
                        stars = int(float(stars_text.replace('k', '')) * 1000)
                    except ValueError:
                        stars = 0
                else:
                    try:
                        # カンマ等が除去された文字列を直接整数に変換
                        stars = int(stars_text)
                    except ValueError:
                        stars = 0
                
                repo_data = {
                    "name": name,
                    "language": language,
                    "stars": stars
                }
                all_repos_data.append(repo_data)

            print(f"    - ページ {page} から {len(repo_items)} 件のリポジトリを収集しました。")
            
            # 次のページへ
            page += 1 

            # サーバーに負荷をかけないための待機時間 (必須要件)
            time.sleep(SLEEP_TIME)
            
        # 🚨 KeyboardInterrupt の捕捉を削除

        except requests.RequestException as e:
            # ネットワークエラーやHTTPエラー（404など）を捕捉し、スクレイピングを中断
            print(f"    ❌ ページ {page} の取得中にエラーが発生しました: {e}")
            break
            
    print(f"✅ スクレイピング処理終了。合計 {len(all_repos_data)} 件のデータを収集しました。")
    return all_repos_data

# ----------------------------------------------------------------------
# 3. データ保存関数
# ----------------------------------------------------------------------

def insert_data_to_db(repos_data: List[Dict[str, Any]]):
    """
    スクレイピングしたデータをデータベースに保存する（一括挿入）。
    """
    if not repos_data:
        print("❌ 保存するデータがありません。")
        return

    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    
    print("▶️ データをデータベースに保存中...")
    
    # データを (name, language, stars) のタプルリストに変換
    data_to_insert = [(repo['name'], repo['language'], repo['stars']) for repo in repos_data]
    
    try:
        # executemany: 複数の行を効率的にデータベースに挿入する
        cursor.executemany("""
            INSERT INTO repositories (name, language, stars) 
            VALUES (?, ?, ?)
        """, data_to_insert)
        
        conn.commit()
        print(f"✅ 全 {len(repos_data)} 件のデータをデータベースに保存しました。")
        
    except sqlite3.Error as e:
        print(f"    ❌ データの挿入中にエラーが発生しました: {e}")

    conn.close()

# ----------------------------------------------------------------------
# 4. データ表示関数
# ----------------------------------------------------------------------

def display_data_from_db():
    """
    データベースからデータを全て取得し、スター数の降順で整形して表示する（全件表示）。
    """
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    
    print("\n▶️ データベースからデータを全て取得し、表示します...")
    
    # SELECT文: 全ての行を取得し、starsの降順でソート
    cursor.execute("""
        SELECT name, language, stars FROM repositories
        ORDER BY stars DESC
    """)
    
    rows = cursor.fetchall()
    conn.close()
    
    if not rows:
        print("    ⚠️ データベースにデータが見つかりませんでした。")
        return

    # 結果の整形表示
    print(f"--- 🌟 Googleリポジトリ ({len(rows)} 件, Star数降順) ---")
    # ヘッダー行
    print(f"{'リポジトリ名':<30} | {'言語':<10} | {'スター数':>8}")
    print("-------------------------------------------------------")
    # データ行
    for name, language, stars in rows:
        # スター数はカンマ区切りで表示（例: 52600 -> 52,600）
        print(f"{name:<30} | {language:<10} | {stars:>8,}")
    print("-------------------------------------------------------")

# ----------------------------------------------------------------------
# 5. メイン処理 (プログラム実行時のエントリーポイント)
# ----------------------------------------------------------------------

if __name__ == '__main__':
    # 1. DBを作成し、初期化する
    setup_database()
    
    # 2. リポジトリがなくなるまで全ページをスクレイピングする
    repo_data = scrape_google_repos() 
    
    # 3. 収集したデータをDBに保存する
    insert_data_to_db(repo_data)
    
    # 4. 保存したデータを全て表示する
    display_data_from_db()

✅ データベース 'github_google_repos.db' を初期化しました。
▶️ GitHub (Googleアカウント) から全リポジトリ情報をスクレイピング中...
    - ページ 1 (https://github.com/google?page=1) にアクセス...
    - ページ 1 から 16 件のリポジトリを収集しました。
    - ページ 2 (https://github.com/google?page=2) にアクセス...
    - ページ 2 から 16 件のリポジトリを収集しました。
    - ページ 3 (https://github.com/google?page=3) にアクセス...
    - ページ 3 から 16 件のリポジトリを収集しました。
    - ページ 4 (https://github.com/google?page=4) にアクセス...
    - ページ 4 から 16 件のリポジトリを収集しました。
    - ページ 5 (https://github.com/google?page=5) にアクセス...
    - ページ 5 から 16 件のリポジトリを収集しました。
    - ページ 6 (https://github.com/google?page=6) にアクセス...
    - ページ 6 から 16 件のリポジトリを収集しました。
    - ページ 7 (https://github.com/google?page=7) にアクセス...
    - ページ 7 から 16 件のリポジトリを収集しました。
    - ページ 8 (https://github.com/google?page=8) にアクセス...
    - ページ 8 から 16 件のリポジトリを収集しました。
    - ページ 9 (https://github.com/google?page=9) にアクセス...
    - ページ 9 から 16 件のリポジトリを収集しました。
    - ページ 10 (https://github.com/google?page=10) にアクセス...
    - ページ 10 から 16 件のリポジトリを収集しました。
    - 